In [ ]:
from inaccel.vitis.fintech import CFBlackScholes, OptionType

In [ ]:
numAssets = 100000

cfBlackScholes = CFBlackScholes(numAssets)

In [ ]:
cfBlackScholes.stockPrice.fill(100.0)
cfBlackScholes.strikePrice.fill(100.0)
cfBlackScholes.volatility.fill(0.1)
cfBlackScholes.riskFreeRate.fill(0.025)
cfBlackScholes.timeToMaturity.fill(1.0)

In [ ]:
cfBlackScholes.run(OptionType.Put, numAssets)

In [ ]:
print("+-------+-----------+----------------+--------------+---------------+---------------+---------------+")
print("| Index | Price     |     Delta      |     Gamma    |     Vega      |     Theta     |     Rho       |")
print("+-------+-----------+----------------+--------------+---------------+---------------+---------------+")

for loop in range(0, 15) :
    print(loop,"\t%9.5f"%cfBlackScholes.optionPrice[loop],"\t%9.5f"%cfBlackScholes.delta[loop],\
          "\t%9.5f"%cfBlackScholes.gamma[loop],"\t%9.5f"%cfBlackScholes.vega[loop],\
          "\t%9.5f"%cfBlackScholes.theta[loop],"\t%9.5f"%cfBlackScholes.rho[loop])

In [ ]:
print("This run took", cfBlackScholes.lastruntime(), "microseconds")

In [ ]:
import math
import numpy as np
import time

In [ ]:
def phi(x):
    x_ret = np.array(x)
    sqrt2 = math.sqrt(2.0)
    for i in range(0, x.size):
        x_ret[i] =  0.5 * math.erfc(-x[i] / sqrt2)
    return x_ret

def CFBlackScholesSW(s,v, r, t, k, q, call):
    d1 = (np.log(s / k) + (r - q + v * v / 2.0) * t) / (v * np.sqrt(t))
    d2 = d1 - v * np.sqrt(t)

    pdf_d1 = (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * d1 * d1)

    if (call):
        price = s * phi(d1) * np.exp(-q * t) - k * phi(d2) * np.exp(-r * t)
        delta = np.exp(-q * t) * phi(d1)
        theta = (1.0 / 365) * (-v * s * np.exp(-q * t) * pdf_d1 / (2 * np.sqrt(t)) + q * s * np.exp(-q * t) * phi(d1) - r * k * np.exp(-r * t) * phi(d2));
        rho = (1.0 / 100) * k * t * np.exp(-r * t) * phi(d2)
    else:
        price = phi(-d2) * k * np.exp(-r * t) - phi(-d1) * s * np.exp(-q * t);
        delta = np.exp(-q * t) * (phi(d1) - 1)
        theta = (1.0 / 365) * (-v * s * np.exp(-q * t) * pdf_d1 / (2 * np.sqrt(t)) - q * s * np.exp(-q * t) * phi(-d1) + r * k * np.exp(-r * t) * phi(-d2))
        rho = (-1.0 / 100) * k * t * np.exp(-r * t) * phi(-d2)

    gamma = np.exp(-q * t) * np.exp(-d1 * d1 / 2) / (s * v * np.sqrt(t) * np.sqrt(2 * np.pi));
    vega = (1.0 / 100) * s * np.exp(-q * t) * np.sqrt(t) * pdf_d1;
    
    return (price, delta, gamma, vega, theta, rho)

In [ ]:
stockPrice = np.array(cfBlackScholes.stockPrice)
strikePrice = np.array(cfBlackScholes.strikePrice)
volatility = np.array(cfBlackScholes.volatility)
riskFreeRate = np.array(cfBlackScholes.riskFreeRate)
timeToMaturity = np.array(cfBlackScholes.timeToMaturity)

optionPrice = np.ndarray(numAssets, dtype = np.float32)
delta = np.ndarray(numAssets, dtype = np.float32)
gamma = np.ndarray(numAssets, dtype = np.float32)
vega = np.ndarray(numAssets, dtype = np.float32)
theta = np.ndarray(numAssets, dtype = np.float32)
rho = np.ndarray(numAssets, dtype = np.float32)

In [ ]:
start = int(round(time.time() * 1000000))

(optionPrice, delta, gamma, vega, theta, rho) = CFBlackScholesSW(stockPrice, volatility, riskFreeRate, timeToMaturity, strikePrice, 0, OptionType.Put)

end = int(round(time.time() * 1000000))

In [ ]:
print("+-------+-----------+----------------+--------------+---------------+---------------+---------------+")
print("| Index | Price     |     Delta      |     Gamma    |     Vega      |     Theta     |     Rho       |")
print("+-------+-----------+----------------+--------------+---------------+---------------+---------------+")

for loop in range(0, 15) :
    print(loop,"\t%9.5f"%optionPrice[loop],"\t%9.5f"%delta[loop], "\t%9.5f"%gamma[loop],"\t%9.5f"%vega[loop],\
          "\t%9.5f"%theta[loop],"\t%9.5f"%rho[loop])

In [ ]:
print("SW run took", end - start, "microseconds")

In [ ]:
print("Speedup: ", (end - start) / cfBlackScholes.lastruntime())